# **WGCNA**

## **Global settings**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.pyplot import rc_context
import scanpy as sc
import scvelo as scv
import scipy.io
import os
import math
import gseapy as gp
from gseapy.plot import barplot, dotplot, gseaplot
from gseapy.scipalette import SciPalette
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap 

In [2]:
sc.settings.set_figure_params(dpi=100, dpi_save=300, figsize=(5, 5))

In [ ]:
os.getcwd()

'/disk213/xieqq/sc'

In [3]:
os.chdir('/disk213/xieqq/sc')

## WGCNA

In [ ]:
# devtools::install_github('smorabit/hdWGCNA', ref='dev')
# devtools::install_local("/disk213/xieqq/sc/WGCNA/smorabit-hdWGCNA-v0.1.1-63-g76da474.tar.gz")

In [ ]:
# single-cell analysis package
library(Seurat)

# plotting and data science packages
library(tidyverse)
library(cowplot)
library(patchwork)

# co-expression network analysis packages:
library(WGCNA)
library(hdWGCNA)

# network analysis & visualization package:
library(igraph)

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

# optionally enable multithreading
enableWGCNAThreads(nThreads = 8)

In [ ]:
setwd("/disk213/xieqq/JINHUA138.sc/WGCNA")
seurat_obj <- readRDS("/disk213/xieqq/JINHUA138.sc/RDS/Epithelial.rds")

In [ ]:
seurat_obj <- subset(seurat_obj, subset = SEGMENT == "duodenum" & TIME == "0")
setwd("/disk213/xieqq/sc/WGCNA/DU_0_E")

In [ ]:
seurat_obj <- SetupForWGCNA(
    seurat_obj,
    gene_select = "fraction",
    fraction = 0.05,
    wgcna_name = "tutorial" # hdWGCNA实验名称
)

In [ ]:
# metacell
seurat_obj <- MetacellsByGroups(
    seurat_obj = seurat_obj,
    group.by = c("CellType","PRO1_JH"),  
    reduction = "harmony", 
    k = 25,
    max_shared = 10, 
    ident.group = "CellType"  
)

seurat_obj <- NormalizeMetacells(seurat_obj)

In [ ]:
head(seurat_obj@misc$tutorial$wgcna_metacell_obj, 2)

In [ ]:
# group_name = "Enterocytes"
group_name = "Colonocytes"
# group_name = c("Enterocytes","Colonocytes","BEST4 enterocytes","EECs","Tuft","Goblet","TA","Stem","Progenitor")

In [ ]:
setwd(paste0("./",group_name))

In [ ]:
seurat_obj <- SetDatExpr(
    seurat_obj,
    group_name = group_name, 
    group.by = "CellType", 
    assay = "RNA",
    slot = "data"
)

In [ ]:
# Soft power
seurat_obj <- TestSoftPowers(
    seurat_obj,
    networkType = "signed" 
)

plot_list <- PlotSoftPowers(seurat_obj)

#wrap_plots(plot_list, ncol=2)

In [ ]:
pdf(file=paste0("Soft_power_",group_name,".pdf"), width=12, height=8)
wrap_plots(plot_list, ncol=2)
dev.off()

In [ ]:
power_table <- GetPowerTable(seurat_obj)
write.csv(power_table, paste0("Soft_power_",group_name,".csv"), row.names =F)

In [ ]:
# construct co-expression network:
seurat_obj <- ConstructNetwork(
    seurat_obj, soft_power = 12, 
    setDatExpr = FALSE,
    tom_name = group_name 
)

#PlotDendrogram(seurat_obj, main='Enterocytes hdWGCNA Dendrogram')

In [ ]:
pdf(file=paste0("hdWGCNA_Dendrogram_",group_name,".pdf"), width=12, height=8)
PlotDendrogram(seurat_obj, main=paste0(group_name," hdWGCNA Dendrogram"))
dev.off()

In [ ]:
seurat_obj@misc$tutorial$wgcna_modules %>% head
table(seurat_obj@misc$tutorial$wgcna_modules$module)
write.csv(seurat_obj@misc$tutorial$wgcna_modules,paste0("wgcna_modules_",group_name,".csv"), row.names =F)
write.csv(table(seurat_obj@misc$tutorial$wgcna_modules$module),paste0("wgcna_modules_count_",group_name,".csv"), row.names =F)

In [ ]:
TOM <- GetTOM(seurat_obj)
write.csv(TOM,paste0("TOM_",group_name,".csv"), row.names =T)

In [ ]:
# Scale
seurat_obj <- ScaleData(seurat_obj, features = VariableFeatures(seurat_obj))

seurat_obj <- ModuleEigengenes(
    seurat_obj,
    group.by.vars = "PRO1_JH"  
)

In [ ]:
# compute eigengene-based connectivity (kME):
seurat_obj <- ModuleConnectivity(
    seurat_obj,
    group.by = 'CellType', 
    group_name = group_name  
)

# module
seurat_obj <- ResetModuleNames(
    seurat_obj,
    new_name = paste0(group_name,"-M")
)

In [ ]:
pdf(file=paste0("hdWGCNA_ModulekME_",group_name,".pdf"), width=24, height=8)
PlotKMEs(seurat_obj, ncol = 4, n_hubs = 10)
dev.off()

In [ ]:
# get the module assignment table:
modules <- GetModules(seurat_obj)

# topN hub
hub_df <- GetHubGenes(seurat_obj = seurat_obj, n_hubs = 10)

saveRDS(seurat_obj, file = paste0("hdWGCNA_object_",group_name,".rds"))

In [ ]:
write.csv(modules,paste0("Module_assignment_",group_name,".csv"))
write.csv(hub_df,paste0("Module_hub_",group_name,".csv"))

In [ ]:
seurat_obj <- ModuleExprScore(
    seurat_obj,
    n_genes = 10, # topN hub genes
    method = "Seurat" # Seurat、UCell
)

In [ ]:
# group_name = "Enterocytes"
group_name = "Colonocytes"

In [ ]:
setwd(paste0("/disk213/xieqq/JINHUA138.sc/WGCNA/",group_name))
seurat_obj <- readRDS(paste0("hdWGCNA_object_",group_name,".rds"))

In [ ]:
gene_set=c("PCK1","APOE")
FeaturePlot(object = seurat_obj, features = gene_set, group.by = "ident", cols = c("blue", "red", "green"))

In [ ]:
# get hMEs from seurat object
MEs <- GetMEs(seurat_obj, harmonized=TRUE)
MEs <- MEs %>% select(sort(names(MEs)))
mods <- colnames(MEs); mods <- mods[mods != 'grey']

# mods=c("Enterocytes-M5","Enterocytes-M1","Enterocytes-M4","Enterocytes-M3","Enterocytes-M7","Enterocytes-M2","Enterocytes-M6")
# mods=c("Colonocytes-M6","Colonocytes-M4","Colonocytes-M5","Colonocytes-M2","Colonocytes-M7","Colonocytes-M1","Colonocytes-M3")

# add hMEs to Seurat meta-data:
seurat_obj@meta.data <- cbind(seurat_obj@meta.data, MEs)
write.csv(MEs,paste0("MEs_",group_name,".csv"))

In [ ]:
plot_list <- ModuleFeaturePlot(
    seurat_obj,
    #module_names = mods, #c("Enterocytes-M2","Enterocytes-M3","Enterocytes-M6","Enterocytes-M7"),
    reduction = "umap",
    features = 'hMEs', # MEs、hMEs、scores、average
    order = TRUE # order so the points with highest hMEs are on top
)

# stitch together with patchwork
wrap_plots(plot_list, ncol=4)

pdf(file=paste0("hdWGCNA_ModulePlot_",group_name,"_hME.pdf"), width=12, height=8)
wrap_plots(plot_list, ncol=4)
dev.off()

In [ ]:
ModuleCorrelogram(seurat_obj)

pdf(file=paste0("hdWGCNA_ModuleCorrelogram_",group_name,".pdf"), width=8, height=8)
ModuleCorrelogram(seurat_obj)
dev.off()

In [ ]:
seurat_obj@meta.data$SEGMENT <- factor(seurat_obj@meta.data$SEGMENT, levels=c("colon","cecum","ileum","jejunum","duodenum"))

p <- DotPlot(seurat_obj, features=mods, group.by="SEGMENT", col.min=0, col.max=2, scale.min=0, scale.max=100) +
     RotatedAxis() + 
     scale_color_gradient(limits=c(0,2),high="#08519C", low="white")
p
ggsave(filename=paste0("hdWGCNA_Module_SEGMENT_",group_name,".pdf"), plot=p, width=8, height=8)

In [ ]:
seurat_obj@meta.data$TIME <- factor(seurat_obj@meta.data$TIME, levels=c("240","180","90","60","0"))

p <- DotPlot(seurat_obj, features=mods, group.by="TIME", col.min=0, col.max=2, scale.min=0, scale.max=100) +
     RotatedAxis() + 
     scale_color_gradient(limits=c(0,2),high="#08519C", low="white")
p
ggsave(filename=paste0("hdWGCNA_Module_TIME_",group_name,".pdf"), plot=p, width=8, height=8)

In [ ]:
seurat_obj@meta.data$SEGMENT.TIME <- factor(seurat_obj@meta.data$SEGMENT.TIME, 
                                            levels=c("colon-240","colon-180","colon-90","colon-60","colon-0",
                                                     "cecum-240","cecum-180","cecum-90","cecum-60","cecum-0",
                                                     "ileum-240","ileum-180","ileum-90","ileum-60","ileum-0",
                                                     "jejunum-240","jejunum-180","jejunum-90","jejunum-60","jejunum-0",
                                                     "duodenum-240","duodenum-180","duodenum-90","duodenum-60","duodenum-0"))

p <- DotPlot(seurat_obj, features=mods, group.by="SEGMENT.TIME", col.min=0, col.max=2, scale.min=0, scale.max=100) +
     RotatedAxis() + 
     scale_color_gradient(limits=c(0,2),high="#08519C", low="white")
p
ggsave(filename=paste0("hdWGCNA_Module_SEG-TIME_",group_name,".pdf"), plot=p, width=8, height=8)

In [ ]:
seurat_obj@meta.data$CellType <- factor(seurat_obj@meta.data$CellType, 
                                        levels=c('Stem','TA','Progenitor','Goblet','Tuft','EECs','BEST4enterocytes','Colonocytes','Enterocytes'),
                                        labels=c('Stem','TA','Progenitor','Goblet','Tuft','EECs','BEST4 enterocytes','Colonocytes','Enterocytes'))

p <- DotPlot(seurat_obj, features=mods, group.by="CellType", col.min=0, col.max=2, scale.min=0, scale.max=100) +
     RotatedAxis() + 
     scale_color_gradient(limits=c(0,2),high="#08519C", low="white")
p
ggsave(filename=paste0("hdWGCNA_Module_CellType_",group_name,".pdf"), plot=p, width=8, height=8)

In [ ]:
My_levels <- c('Enterocytes','Colonocytes','BEST4 enterocytes','EECs','Tuft','Goblet','Progenitor','TA','Stem')
seurat_obj$CellType <- factor(seurat_obj$CellType,levels=My_levels)
My_colors <- c('#004B23','#007200','#38B000','#52B788','#95D5B2','#D8F3DC','#34A0A4','#1A759F','#184E77')

In [ ]:
plot_list <- lapply(mods, function(x) {
  print(x)
  p <- VlnPlot(
    seurat_obj,
    features = x,
    group.by = 'CellType',
    pt.size = 0 # don't show actual data points
  )
  p <- p + geom_boxplot(width = .25, fill = "white")+xlab("") + ylab("hME") + NoLegend() + scale_fill_manual(values = My_colors)
  p
})

wrap_plots(plot_list, ncol = 4)

In [ ]:
pdf(file=paste0("hdWGCNA_violin_CellType_",group_name,".pdf"), width=12, height=8)
wrap_plots(plot_list, ncol = 4)
dev.off()

In [ ]:
p <- VlnPlot(
    seurat_obj,
    features = "Enterocytes-M2",
    group.by = "CellType",
    pt.size = 0
)
p <- p + geom_boxplot(width = .25, fill = "white")+xlab("") + ylab("hME") + NoLegend()
p

In [ ]:
FeaturePlot(seurat_obj, features=mods, ncol=4)

In [ ]:
pdf(file=paste0("hdWGCNA_umap_CellType_",group_name,".pdf"), width=16, height=8)
FeaturePlot(seurat_obj, features=mods, ncol=4)
dev.off()

In [ ]:
# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)  

# ModuleNetworkPlot(seurat_obj = seurat_obj, mods = "Colonocytes-M2")

In [ ]:
pdf(file=paste0("hdWGCNA_ModuleNetworkPlot_all_",group_name,".pdf"), width=8, height=8)
HubGeneNetworkPlot(
    seurat_obj,
    n_hubs = 10, 
    n_other = 5, 
    edge_prop = 0.75,
    mods = "all"
)
dev.off()

In [ ]:
seurat_obj$TIME <- factor(seurat_obj$TIME,levels=c("240","180","90","60","0"))
seurat_obj$TIME <- as.factor(seurat_obj$TIME)
seurat_obj$SEGMENT <- factor(seurat_obj$SEGMENT,levels=c("colon","cecum","ileum","jejunum","duodenum"))
seurat_obj$SEGMENT <- as.factor(seurat_obj$SEGMENT)

cur_traits <- c('TIME','SEGMENT')

seurat_obj <- ModuleTraitCorrelation(
  seurat_obj,
  traits = cur_traits #, 
  #group.by = 'CellType'
)
mt_cor <- GetModuleTraitCorrelation(seurat_obj)
names(mt_cor$cor)
mt_cor$cor$all_cells

write.csv(mt_cor$cor$all_cells, paste0("ModuleTraitCorrelation_",group_name,".csv"), row.names =T)
write.csv(mt_cor$pval$all_cells, paste0("ModuleTraitCorrelationPval_",group_name,".csv"), row.names =T)

In [ ]:
pdf(file=paste0("ModuleTraitCorrelation_",group_name,".pdf"), width=10, height=10)
PlotModuleTraitCorrelation(
  seurat_obj,
  label = 'pval',  
  label_symbol = 'stars',  
  text_size = 2,
  text_digits = 2,
  text_color = "black",
  high_color = "#CB1B16",
  mid_color = "white",
  low_color = "#1368AA",
  plot_max = 0.2,
  combine=TRUE
)
dev.off()

## Enrichment

In [3]:
os.chdir('/disk213/xieqq/sc/WGCNA_Enrich/Enterocytes')

### GO

In [ ]:
modulecolor = ['blue', 'yellow', 'green', 'red', 'brown', 'turquoise', 'black'] 

In [ ]:
for i in modulecolor[:]:
    filename=i
    gene_list = data[data['color'].isin([i])]
    enr = gp.enrichr(gene_list=gene_list['gene_name_DAVID'],
                     gene_sets='GO_Biological_Process_2023',  #['GO_Biological_Process_2023','GO_Cellular_Component_2023','GO_Molecular_Function_2023']
                     organism='Human',
                     outdir='./',
                     cutoff=0.05,  #Wilcoxon test+FDR
                     no_plot=True
                    )
    enr.results = enr.results[enr.results['Adjusted P-value'] < 0.05]
    enr.results['Group'] = filename.replace('Epithelial_enrich_', '')
    enr.results['-Log10 P-value'] = -enr.results['Adjusted P-value'].apply(math.log10)
    enr.results['Count'] = enr.results['Overlap'].map(lambda x:x.split('/')[0])
    enr.results['Gene Count'] = gene_list.shape[0]
    enr.results['Background Count'] = enr.results['Overlap'].map(lambda x:x.split('/')[1])
    enr.results['Count'] = pd.to_numeric(enr.results['Count'],errors='coerce')
    enr.results['Gene Count'] = pd.to_numeric(enr.results['Gene Count'],errors='coerce')
    enr.results['Background Count'] = pd.to_numeric(enr.results['Background Count'],errors='coerce')
    enr.results['Fold Enrichment'] = (enr.results['Count']/enr.results['Gene Count'])/(enr.results['Background Count']/14937)
    
    enr.results=enr.results.sort_values(by='Fold Enrichment', ascending=False)
    enr.results=enr.results.reset_index(drop = True)
    enr.results.to_csv(filename+'_GO.csv')

In [ ]:
files = os.listdir('./')

def file_filter(f):
    if f[-7:] in ['_GO.csv']:
        return True
    else:
        return False

files = list(filter(file_filter, files))
print(files)

In [55]:
data_list = []
for i in files:
    tmp = pd.read_csv(i, index_col=0)
    data_list.append(tmp)
dataset = pd.concat(data_list,ignore_index = False)
dataset = dataset.reset_index(drop = True)
dataset.to_csv('WGCNA_enrich_GO.csv')

In [56]:
data_list = []
for i in files:
    tmp = pd.read_csv(i, index_col=0)
    tmp = tmp.head(5)
    data_list.append(tmp)
dataset = pd.concat(data_list,ignore_index = False)
dataset = dataset.reset_index(drop = True)
dataset.to_csv('WGCNA_enrich_GO_top5.csv')

### KEGG

In [ ]:
for i in modulecolor[:]:
    filename=i
    gene_list = data[data['color'].isin([i])]
    enr = gp.enrichr(gene_list=gene_list['gene_name_DAVID'],
                     gene_sets='KEGG_2021_Human',
                     organism='Human',
                     outdir='./',
                     cutoff=0.05,  #Wilcoxon test+FDR
                     no_plot=True
                    )
    enr.results = enr.results[enr.results['Adjusted P-value'] < 0.05]
    enr.results['Group'] = filename.replace('Epithelial_enrich_', '')
    enr.results['-Log10 P-value'] = -enr.results['Adjusted P-value'].apply(math.log10)
    enr.results['Count'] = enr.results['Overlap'].map(lambda x:x.split('/')[0])
    enr.results['Gene Count'] = gene_list.shape[0]
    enr.results['Background Count'] = enr.results['Overlap'].map(lambda x:x.split('/')[1])
    enr.results['Count'] = pd.to_numeric(enr.results['Count'],errors='coerce')
    enr.results['Gene Count'] = pd.to_numeric(enr.results['Gene Count'],errors='coerce')
    enr.results['Background Count'] = pd.to_numeric(enr.results['Background Count'],errors='coerce')
    enr.results['Fold Enrichment'] = (enr.results['Count']/enr.results['Gene Count'])/(enr.results['Background Count']/8078)
    
    enr.results=enr.results.sort_values(by='Fold Enrichment', ascending=False)
    enr.results=enr.results.reset_index(drop = True)
    enr.results.to_csv(filename+'_KEGG.csv')

In [ ]:
files = os.listdir('./')

def file_filter(f):
    if f[-9:] in ['_KEGG.csv']:
        return True
    else:
        return False

files = list(filter(file_filter, files))
print(files)

In [59]:
data_list = []
for i in files:
    tmp = pd.read_csv(i, index_col=0)
    data_list.append(tmp)
dataset = pd.concat(data_list,ignore_index = False)
dataset = dataset.reset_index(drop = True)
dataset.to_csv('WGCNA_enrich_KEGG.csv')

In [60]:
data_list = []
for i in files:
    tmp = pd.read_csv(i, index_col=0)
    tmp = tmp.head(5)
    data_list.append(tmp)
dataset = pd.concat(data_list,ignore_index = False)
dataset = dataset.reset_index(drop = True)
dataset.to_csv('WGCNA_enrich_KEGG_top5.csv')